In [327]:
import pandas as pd
import panel as pn

pn.extension()
import hvplot as hv
import hvplot.pandas
import holoviews as hv
from holoviews import opts# noqa

In [328]:
data = pd.read_csv('a-Liste des adhérents_cleaned.csv')

In [329]:
data.head()

,Métropole,Date d'adhésion,Prénom,NOM,nb Adhérents,indiv,famille,Nom des membres famille,Adresse mail,Lieu de résidence,...,préparation et tenue de stand,élaboration de supports de communication,veille sur l'actualité du vélo,amélioration de la réglementation,atelier de réparation,cartographie collaborative,"autre, précisez.2",Unnamed: 43,Unnamed: 44,Vos attentes
0,x,21/03/2014,Manuel,LEGER,1.0,NaN,NaN,NaN,leger_manuel@yahoo.fr,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,x,21/03/2014,Emmanuelle,SAILLY,1.0,NaN,NaN,NaN,NaN,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,29/03/2014,Luc,MICHEL,1.0,NaN,NaN,NaN,mail supprimé à la demande de Luc Michel,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voies protégées. Vélos électriques à prix rais...
3,NaN,29/03/2014,Michèle,JOHN,1.0,NaN,NaN,NaN,michele.c.john@gmail.com,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,29/03/2014,Daniel,BRETAUDEAU,1.0,NaN,NaN,NaN,danielbretaudeau@gmail.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [330]:
data["date_adhesion"] = pd.to_datetime(data["Date d'adhésion"],format='%d/%m/%Y')


In [331]:
data.loc[data['nb Adhérents'] > 1, "montant_adhesion"] = '4000'
data.loc[data['nb Adhérents'] == 1, "montant_adhesion"] = '2000'

data.head()

,Métropole,Date d'adhésion,Prénom,NOM,nb Adhérents,indiv,famille,Nom des membres famille,Adresse mail,Lieu de résidence,...,veille sur l'actualité du vélo,amélioration de la réglementation,atelier de réparation,cartographie collaborative,"autre, précisez.2",Unnamed: 43,Unnamed: 44,Vos attentes,date_adhesion,montant_adhesion
0,x,21/03/2014,Manuel,LEGER,1.0,NaN,NaN,NaN,leger_manuel@yahoo.fr,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-21,2000
1,x,21/03/2014,Emmanuelle,SAILLY,1.0,NaN,NaN,NaN,NaN,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-21,2000
2,NaN,29/03/2014,Luc,MICHEL,1.0,NaN,NaN,NaN,mail supprimé à la demande de Luc Michel,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Voies protégées. Vélos électriques à prix rais...,2014-03-29,2000
3,NaN,29/03/2014,Michèle,JOHN,1.0,NaN,NaN,NaN,michele.c.john@gmail.com,Nouméa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-29,2000
4,NaN,29/03/2014,Daniel,BRETAUDEAU,1.0,NaN,NaN,NaN,danielbretaudeau@gmail.com,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-29,2000


In [332]:
row_slider = pn.widgets.IntSlider(value=data["date_adhesion"].min().year, start=data["date_adhesion"].min().year,end=data["date_adhesion"].max().year )
row_slider

IntSlider(end=2020, start=2014, value=2014)

In [333]:
@pn.depends(row_slider.param.value)
def select_date(row=data["date_adhesion"].min().year):
    return data.loc[data["date_adhesion"].dt.year == row]


app = pn.Row(row_slider, select_date)

In [334]:
def getDfYear(year):
    filter = data["date_adhesion"].dt.year == year
    df_year = data[filter]
    return df_year

def bar_adherent(df):
    bar = df.groupby(['d']).hvplot.bar(y="")
    return bar
    
d_2020 = pd.concat([getDfYear(2019),  getDfYear(2020)])

In [335]:
famille_2020 = d_2020.loc[(d_2020["famille"]==1)]
individuel_2020 = d_2020.loc[(d_2020["indiv"]==1)]
adhesion_indiv = len(individuel_2020)
adhesion_famille = len(famille_2020)
montant_tresor = adhesion_famille * 4000 + adhesion_indiv *2000
nbAdherent_famille = famille_2020["nb Adhérents"].sum()
nbAdherent = len(individuel_2020) + nbAdherent_famille
nb_adhesion = adhesion_famille + adhesion_indiv

In [336]:
rapport_fammille_indiv = adhesion_famille/nb_adhesion

In [337]:
rapport_fammille_indiv

0.3382352941176471

In [338]:
nbAdherent_famille

75.0

In [339]:
nbAdherent

120.0

# Création du modèle

## Créer la typologie des familles

In [340]:
famille_2020_sorted = famille_2020.sort_values(by=["nb Adhérents"])
df_type_famille = famille_2020_sorted.groupby(['nb Adhérents'])
bar = df_type_famille.size().hvplot.bar().opts(title="Nombre de famille selon le nombre d'adhérent",ylabel="nb famille")
moyenne =famille_2020["nb Adhérents"].mean()

moyenne_line = hv.VLine(moyenne-1)
layout = bar * moyenne_line


In [341]:
layout.opts(opts.VLine(color='red', line_width=4))

:Overlay
   .Bars.A_0 :Bars   [nb Adhérents]   (0)
   .VLine.I  :VLine   [x,y]

### défintion des variables

In [427]:
rapport_fammille_indiv = 0.3
montant_adhésion_famille = 3000

In [428]:
dfm = pd.DataFrame({'nb_adherents': range(1,200)})

In [429]:
dfm['tresor_4000'] = dfm['nb_adherents']*4000

In [430]:
dfm['tresor_3000'] = dfm['nb_adherents']*3000

In [431]:
dfm['tresor_2000'] = dfm['nb_adherents']*2000

In [432]:
dfm["nb_adhesion"] =  dfm['nb_adherents'] * moyenne

In [433]:
dfm["nb_adhesion_calc"] = dfm["nb_adhesion"] * rapport_fammille_indiv

In [434]:
dfm["tresor_calc_ratio_famille"] = round(((dfm['nb_adherents'] * rapport_fammille_indiv )*montant_adhésion_famille)+ (dfm['nb_adherents'] *(1- rapport_fammille_indiv)* 2000 ))

In [455]:
row = dfm.iloc[nb_adhesion]

In [435]:

def formatter(value):
    return str(value) 

In [436]:
nbAdherent_famille


75.0

In [459]:
nbAdherent_line = hv.VLine(nbAdherent).opts(alpha=0.5,color='grey', line_width=2,line_dash='dashed' )

famille_points = hv.Points((adhesion_famille,adhesion_famille*4000)).opts( size=8)
indiv_point = hv.Points((adhesion_indiv,adhesion_indiv*2000)).opts(color='green', size=8)
famille_adherents_points = hv.Points((nbAdherent_famille,adhesion_famille*4000)).opts(size=8)

total_point = hv.Points((nb_adhesion,montant_tresor)).opts(color='grey', size=8)

famille_Hline = hv.HLine(adhesion_famille*4000).opts(alpha=0.3,color='blue',line_width=1)


layout = (dfm.hvplot.line(x="nb_adherents", y="tresor_4000") *
 dfm.hvplot.line(x="nb_adherents", y="tresor_3000") * 
 dfm.hvplot.line(x="nb_adherents", y="tresor_2000") * 
 dfm.hvplot.line(x="nb_adherents", y="tresor_calc_ratio_famille") * 

nbAdherent_line *
          famille_points *
          indiv_point * 
          total_point* 
         famille_Hline).opts(xlabel="Nb adhésions", ylabel="Trésor",yformatter=formatter,ylim=(0,500000),xlim=(0,150),show_grid=True)
layout

:Overlay
   .Curve.I    :Curve   [nb_adherents]   (tresor_4000)
   .Curve.II   :Curve   [nb_adherents]   (tresor_3000)
   .Curve.III  :Curve   [nb_adherents]   (tresor_2000)
   .Curve.IV   :Curve   [nb_adherents]   (tresor_calc_ratio_famille)
   .VLine.I    :VLine   [x,y]
   .Points.I   :Points   [x,y]
   .Points.II  :Points   [x,y]
   .Points.III :Points   [x,y]
   .HLine.I    :HLine   [x,y]

In [460]:

print(row["nb_adhesion"])


225.0


In [438]:

nb_adherent_r100_line = dfm.hvplot.line(x="nb_adherents", y="nb_adhesion")
dynamique_curve = dfm.hvplot.line(x="nb_adherents", y="nb_adhesion_calc")
nb_adherent_r1_line = dfm.hvplot.line(x="nb_adherents", y="nb_adherents")


layout_ratio = (nb_adherent_r100_line * dynamique_curve *nb_adherent_r1_line
).opts(xlabel="Nb adhérents", ylabel="Nb adhésions",yformatter=formatter,ylim=(0,500),xlim=(0,150),show_grid=True)

layout_ratio

:Overlay
   .Curve.I   :Curve   [nb_adherents]   (nb_adhesion)
   .Curve.II  :Curve   [nb_adherents]   (nb_adhesion_calc)
   .Curve.III :Curve   [nb_adherents]   (nb_adherents)